In [1]:
import numpy as np
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import optuna
from optuna.samplers import RandomSampler
import warnings
warnings.filterwarnings("ignore")
import multiprocessing
max_n_jobs = multiprocessing.cpu_count()
print(f"Maximum n_jobs you can use: {max_n_jobs}")

Maximum n_jobs you can use: 12


/Users/justpqa/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
feature_version = 1
default_cv = 1
# 1 for pc feature, 
# 2 for label correlation feature
# 3 for best features based on combination rank

Train the best model

In [3]:
train_df = pd.read_parquet(f"cleaned_train_{feature_version}.parquet")
X_train = train_df.drop(columns=["timestamp", "label"])
Y_train = train_df["label"]

In [4]:
default_n_trees = 1000
default_random_state = 101

In [5]:
def get_best_params_from_file(filename):
    study = optuna.load_study(
        study_name = filename,
        storage = f"sqlite:///{filename}.db"
    )
    return study.best_params

In [6]:
params = {
    "n_estimators": default_n_trees,
    "verbosity": 0,
    "enable_categorical": True,
    "random_state": default_random_state
}
best_params_xgboost = get_best_params_from_file(f"xgboost_{feature_version}_{default_cv}_{default_random_state}_{default_n_trees}_study")
for p in best_params_xgboost:
    params[p] = best_params_xgboost[p]

feature_importances = {}

xgbr = XGBRegressor(**params)
xgbr.fit(X_train, Y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=0.864175007009998,
             colsample_bynode=0.02219539910069701,
             colsample_bytree=0.3251453256519865, device=None,
             early_stopping_rounds=None, enable_categorical=True,
             eval_metric=None, feature_types=None, gamma=3.2392546763596113,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.013982300816904816,
             max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1000, n_jobs=None,
             num_parallel_tree=None, random_state=101, ...)

Making prediction & submission

In [7]:
X_test = pd.read_parquet(f"cleaned_test_{feature_version}.parquet")
X_test = X_test.drop(columns=["label"])
Y_pred = xgbr.predict(X_test)

In [8]:
submission = pd.DataFrame({
    "id": X_test.index + 1,
    "prediction": Y_pred
})
submission.head()

,id,prediction
0,1,0.156244
1,2,-0.010466
2,3,0.026512
3,4,-0.001109
4,5,0.063876


In [9]:
submission.to_csv('submission.csv', index=False)